# Generative AI Dynamic Prompt Management 

Generative AI applications have revolutionized how we interact with technology, but their effectiveness hinges on a critical factor: up-to-date context. In this tutorial, we'll explore why delivering current and relevant information is crucial for AI models to produce accurate, useful, and timely outputs. We'll examine how missing or outdated context can lead to irrelevant or incorrect responses, and diminished user trust. 

This tutorial focuses on using Tecton to create and manage context enriched generative AI prompts. Tecton provides feature pipelines that deliver the required data freshness for batch, streaming and real-time data flows at scale. You'll start with a Tecton workspace that has multiple feature pipelines are already deployed:

- restaurant features, 
- user's profile features
- user's last 100 restaurants visited
- user's ratings by cuisine

In this tutorial you will:
- create a simple AI chat function to simulate a gen AI application
- design a prompt for a restaurant recommendation app
- refine the prompt by adding context from real-time features 
- create a new version of the prompt and setup A/B testing
- visualize lineage of each prompt version 
- use time travel to test the prompts against context in the past



In [ ]:
!pip install 'tecton[rift]==0.9.0' gcsfs s3fs -q

# Initialization

In [ ]:
import tecton, pandas as pd
from tecton import *
from tecton.types import *
from datetime import datetime, timedelta
from pprint import pprint

tecton.login('community.tecton.ai')

tecton.set_validation_mode('auto')
tecton.conf.set('TECTON_OFFLINE_RETRIEVAL_COMPUTE_MODE', 'rift')
tecton.conf.set('TECTON_BATCH_COMPUTE_MODE', 'rift')

### Install openai 

In [ ]:
pip install --upgrade openai

### Create a simple AI Chat Function

The following cell creates a simple gen AI function using GPT-4o. It takes the user prompt and the system prompt and provides a single step chat response.

For this step you will need to:
- Obtain an [OpenAI API key](https://platform.openai.com/api-keys) and replace "your-openai-key" in the following cell.
- Retrieve your [OpenAI Organization Id](https://platform.openai.com/settings/organization/general) and replace "your-organization-id" in the following cell.


In [ ]:
# setup simple ai chat
import openai as oa

openai_api_key = "your-openai-key"
openai_org_id = "your-organization-id"

# very simple chat function inspired by _j at https://community.openai.com/t/how-do-i-call-chatgpt-api-with-python-code/554554/2
def ai_chat( user_prompt:str, system_prompt: str="" )  :
    cl=oa.OpenAI(
        api_key = openai_api_key,
        organization = openai_org_id
        )
    
    messages =[] 
    messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": user_prompt })

    cc=cl.chat.completions.create(
        messages=messages,
        model="gpt-4o")
    return "".join([s.message.content for s in cc.choices]).replace("**","\n")
    
    

## Open AI Test without Context

This first test shows how a system prompt without user context, does not have enough information to provide a good response.

In [ ]:
system_context = """
You are a consierge service that recommends restaurants. 
Your response always includes at least one specific restaurant recommendation 
Also suggests menu items from the recommended restaurant. 
Provide the address for the restaurant you recommend.
"""
response = ai_chat("what should I eat tonight?", system_context)

print(response)

The response it gives you will vary but it will likely be nowhere near your location or have any sense of what kind of food you like. 
I'm located in Charlotte, North Carolina, and the response for me was:
```
    Il Forno
    Address: 500 E Main St, Columbus, OH 43215
    ...
```

Clearly nowhere near me, but I do like Italian.

## Tecton Managed Prompts with Context

A Tecton on-demand feature view is a great construct for creating up-to-date context for generative AI applications. It provides:
- Readily accessible features to create context with batch, stream and real-time data freshness
- Full lineage - providing a source for dependency and impact analysis
- Version controlled prompts integrated with your coding repository best practices

The following cell creates an example of a feature enriched Tecton Prompt, it adds real-time context by including the user's current location and the time of day to the prompt: 

In [ ]:
#ODFV for the feature enhanced prompt 
# add location and calculated time of day as context for LLM

from tecton import RequestSource, on_demand_feature_view
from tecton.types import String, Timestamp, Float64, Field, Bool, String

# request data source obtains the user's current location
request_schema = [Field("location", String)]
user_recommendation_request = RequestSource(schema=request_schema)

# on-demand feature view that constructs the system prompt with location and time as context
@on_demand_feature_view(
    name="restaurant_recommender_prompt:v1", # variant nomenclature provides versioning
    sources=[user_recommendation_request],   # identifies sources of features
    mode="python",
    schema=[Field("system_prompt", String)],
    description="Contextual prompt for recommending a restaurant",
)
def restaurant_recommender_prompt(user_recommendation_request):
    
    system_prompt_template = """
        You are a consierge service that recommends restaurants. 
        Your response always includes at least one specific restaurant recommendation 
        Also suggests menu items from the recommended restaurant. 
        Provide the address for the restaurant you recommend.
        Recommend places that are close to {location}.
        Unless prompted by the user, provide choices appropriate for this time: {time}
        You treat them like a good friend."
    """
    # build dynamic context into the prompt
    prompt_with_context = system_prompt_template.format(
        location=user_recommendation_request["location"],
        time=datetime.now().strftime("%H:%M")
    )
    return {"system_prompt": prompt_with_context}

restaurant_recommender_prompt.validate()

In the following cell you simulate the mobile app's request time data adding the location expressed in "latitude=X,longitude=Y". Test it with your own location to see how the recommendation changes.
This cell is also an example of how the prompt development process can be tested, by adjusting the prompt in the cell above and iteratively testing the results.

In [ ]:
# the mobile app obtains GPS location and provides it to the app
data = {"user_recommendation_request":{"location":"latitude=35.0,longitude=-80.0"}}

#test the prompt generation directly from the SDK
prompt = restaurant_recommender_prompt.run_transformation(input_data=data)

response = ai_chat("what should I eat?", prompt['system_prompt'])
print(response)

Now it recommends a restaurant that is near the location I provided and it also reflects the time of day. In my case, it recommended a place for lunch given that I tested at 11:38AM local time.
My result:
```
"The Fig Tree Restaurant."
Address
: 1601 E 7th St, Charlotte, NC 28204
...
```

## Using Available Features for Better Context
So far you've added real-time context to the prompt by using a request time data source for location and calculated local time in the code of the prompt. In this section we'll add more context about the user's preferences based on the user's own history of restaurant ratings.

In the following cell you connect to the Tecton Workspace and list the available feature views. During prompt development you can make use of any of the available feature views to easily provide additional context.

In [ ]:
# list deployed feature views 
ws = tecton.get_workspace('gen-ai-prompt')
ws.list_feature_views()

You can see access any of the feature views on the platform with `tecton.get_feature_view` and see what specific features are available in the feature view by using the `<feature view>.get_feature_columns:

In [ ]:
#import the feature view we want from the Tecton workspace
user_ratings_and_total_visits_by_cuisine_raw = tecton.get_feature_view(name='user_ratings_and_total_visits_by_cuisine_raw', workspace='gen-ai-prompt')
user_ratings_and_total_visits_by_cuisine_raw.get_feature_columns()

## Creating a Second Version of the Prompt
For the next revision of the prompt, you will add the user's ratings by cuisine to the context allowing the application to make recommendations based on the user's preferences:

In [ ]:
#Create a variant ODFV for feature enhanced prompt with additional knowledge of the user
@on_demand_feature_view(
    name="restaurant_recommender_prompt:v2", #variant name for versioning
    sources=[user_recommendation_request,                   # still including current location 
             user_ratings_and_total_visits_by_cuisine_raw], # add ratings by cuisine
    mode="python",
    schema=[Field("system_prompt", String)],
    description="Contextual prompt for recommending a restaurant",
)
def restaurant_recommender_prompt_v2(user_recommendation_request, user_ratings_and_total_visits_by_cuisine_raw):
    
    system_prompt_template = """
        You are a consierge service that recommends restaurants. 
        Your response always includes at least one specific restaurant recommendation and suggests menu items. 
        Provide the address for the restaurant you recommend.
        Recommend places that are close to {location}.
        If they don't provide a cuisine choose one based on this data:
        {cuisine_data}
        Unless prompted by the user, provide choices appropriate for this time: {time}

        You treat them like a good friend."
    """

    # use preaggregated values from user's rating history
    cuisines = user_ratings_and_total_visits_by_cuisine_raw['cuisine_keys_1825d']
    ratings = user_ratings_and_total_visits_by_cuisine_raw['users_average_rating_by_cuisine_last_5y']
    visits = user_ratings_and_total_visits_by_cuisine_raw['users_total_visits_by_cuisine_last_5y']
    
    # format the ratings for the prompt
    cuisine_ratings = []
    for cuisine, rating, visit in zip(cuisines, ratings, visits):
        cuisine_ratings.append({
            'cuisine': cuisine,
            'average_rating': rating,
            'total_visits': visit
        })
    
    #build final prompt with location, time and cuisine context
    prompt_with_context = system_prompt_template.format(
        location = user_recommendation_request["location"],
        time = datetime.now().strftime("%H:%M"),
        cuisine_data = str(cuisine_ratings)
    )

    return {"system_prompt": prompt_with_context}

restaurant_recommender_prompt_v2.validate()

## Lineage

You can take a look at the Tecton UI to see the different metadata and pipelines of each version of the prompt at:

[Pipeline view for prompt `restaurant_recommender_prompt:v1`](https://community.tecton.ai/app/repo/gen-ai-prompt/feature-services/prompt_restaurant_recommender%3Av1/pipeline)
Screenshot:
![](./prompt-v1-pipeline.png)
The V1 pipelines shows the dependency on the real-time request data source, the transformations it uses to produce the context enriched prompt and the online/batch service that serves it.


[Pipeline view for prompt `restaurant_recommender_prompt:v2`](https://community.tecton.ai/app/repo/gen-ai-prompt/feature-services/prompt_restaurant_recommender%3Av2/pipeline)
Screenshot:
![](./prompt-v2-pipeline.png)
The V2 pipelines starts earlier, showing the retaurants and user ratings activity as sources that are transformed into 3 features that keep the last 5 years average rating by cuisine, it also shows the dependency on the real-time request data source, the transformations it uses to produce the context enriched prompt from both the batch and the request time data source and the online/batch service that serves it.


### Setup tecton_client for feature retrieval

You will need to 
- create a [Tecton client API Key by clicking "Create Service Account" here](https://community.tecton.ai/app/settings/accounts-and-access/service-accounts) 
- use it in the following cell replacing "your-api-key" for your key.

The following cells install "tecton_client" package and instantiate a `tecton_client` object for retrieval from a feature service.

In [ ]:
pip install tecton_client

In [ ]:
from tecton_client import TectonClient

tecton_client = TectonClient(url="https://community.tecton.ai/", 
                             api_key = "your-api-key"  # REPLACE WITH YOUR TECTON API KEY
                             default_workspace_name="gen-ai-prompt")



### Test Restaurant Recommendation App consuming from the Feature Service

The following cell simulates the restaurant recommender app by identifying the `user_entity`, and providing the location information in real-time through `app_data`. It uses the `tecton_client` to request online features from the feature service.

In [ ]:
# simulate a specific user of the app 
user_id = '953c6db5-89ea-4189-b286-c662591487c8'
user_entity = {'user_id': user_id}

# simulate request data from mobile device GPS location
app_data = {"location" : "latitude=35.0,longitude=-80.0"}  

features = tecton_client.get_features(
    feature_service_name="prompt_restaurant_recommender:v2",
    join_key_map=user_entity,
    request_context_map=app_data,
).get_features_dict()

system_prompt = features['restaurant_recommender_prompt:v2.system_prompt']
print("DYNAMIC SYSTEM PROMPT:\n================================================\n")
print(system_prompt)
print("\n================================================\n CHAT RESPONSE:\n")

response = ai_chat("what should I eat?", system_prompt)
print(response)


It provides a recommendation based on the cuisine data that we provided. Notice that if the user asks for a specific cuisine, the response changes accordingly:

In [ ]:
response = ai_chat("I feel like eating Indian food, where should I eat?", prompt['system_prompt'])
print(response)

In [ ]:
from datetime import datetime
#testing with time travel
chat_events = [
        {"timestamp":datetime.strptime("2021-10-01 09:00:01-05:00","%Y-%m-%d %H:%M:%S%z"),"user_id":user_id, "user_message": "where should I eat?", "location":"latitude=35.0,longitude=-80.0"},
        {"timestamp":datetime.strptime("2022-10-01 09:00:01-05:00","%Y-%m-%d %H:%M:%S%z"),"user_id":user_id, "user_message": "where should I eat?", "location":"latitude=35.0,longitude=-80.0"},
        {"timestamp":datetime.strptime("2023-10-01 09:00:01-05:00","%Y-%m-%d %H:%M:%S%z"),"user_id":user_id, "user_message": "where should I eat?", "location":"latitude=35.0,longitude=-80.0"},
        ]
events_df = pd.DataFrame(chat_events)

ws = tecton.get_workspace("gen-ai-prompt")
fs = ws.get_feature_service("prompt_restaurant_recommender:v2")
batch_data = fs.get_features_for_events(events_df, timestamp_key="timestamp")


In [ ]:
for prompt in batch_data.to_pandas()["restaurant_recommender_prompt:v2__system_prompt"]:
    print (prompt)

# Conclusions

Context enriched prompts improve generative AI applications by providing relevant context. There are many benefits to using the Tecton platform to develop and manage prompts, 
- Bring standardization and DevOps best practices to your mission critical prompts.
- Enrich your prompts with features for personalized LLM responses.
- Unlock fine tuning and backtesting with historical prompt accuracy.
- Enable versioning, discoverability, and lineage for prompts.
- Iterate rapidly and A/B test prompts without depending on application engineers.
